# Библиотеки

In [ ]:
!pip install pymorphy2 -q

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install numpy==1.23 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 28.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
pyldavis 3.4.1 requires numpy>=1.24.2, but you have numpy 1.23.0 which is incompatible.
tensorflow 2.14.0 requires numpy>=1.23.5, but you have numpy 1.23.0 which is incompatible.


In [ ]:
!pip install pyLDAvis -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1.26.2 which is incompatible.


In [ ]:
!pip install geopy -q
#!pip install nltk -q
!pip install pandas==1.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 20.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 1.5.1 which is incompatible.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.1 which is incompatible.


In [ ]:
import pandas as pd
from collections import defaultdict
from geopy.geocoders import Nominatim
from geopy import Yandex     # на будущее, чтобы ошибки исправлять
from geopy import BANFrance  # только франция
import numpy as np
import time

#import pymorphy2
import nltk
from nltk.corpus import stopwords
import re
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/Авалон/Диплом')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Данные

## Загрузка

In [ ]:
df = pd.read_csv('data/data.csv')
df['rating'] = df['rating'].astype('float16')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(500000, 5)

## Создание полезных словарей

In [ ]:
# для подсчета количества фирм в каждой рубрике
# составление словаря рубрика - компании в этой рубрике
rubric_set_firm = defaultdict(set)
for rub_all_x, name_x in zip(df['rubrics'], df['name_ru']):
    for rub_x in set(rub_all_x.split(";")):
        rubric_set_firm[rub_x].add(name_x)

df_tmp = pd.DataFrame(rubric_set_firm.items())
df_tmp.columns = ['Рубрика', 'Кол-во фирм']
df_tmp['Кол-во фирм'] = df_tmp['Кол-во фирм'].apply(lambda x: len(x))
df_tmp = df_tmp.set_index('Кол-во фирм').sort_index(ascending=False)
df_tmp.head(10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Рубрика
Кол-во фирм,
15421,Кафе
10860,Гостиница
10804,Ресторан
7430,Салон красоты
5637,"Автосервис, автотехцентр"
5397,"Бар, паб"
4803,Ногтевая студия
4537,Быстрое питание
3906,Доставка еды и обедов


In [ ]:
# для подсчета количества отзывоа по каждой рубрике
# составление словаря рубрика: [фирма, фирма, ..., фирма]
rubric_name_list = defaultdict(list)
for rub_all_x, name_x in zip(df['rubrics'], df['name_ru']):
    for rub_x in set(rub_all_x.split(";")):
        rubric_name_list[rub_x].append(name_x)

df_tmp = pd.DataFrame(rubric_name_list.items())
df_tmp.columns = ['Рубрика', 'Кол-во отзывов']
df_tmp['Кол-во отзывов'] = df_tmp['Кол-во отзывов'].apply(lambda x: len(x))
df_tmp = df_tmp.set_index('Кол-во отзывов').sort_index(ascending=False)
df_tmp.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Рубрика
Кол-во отзывов,
58496,Кафе
56761,Ресторан
43133,Гостиница
21346,Магазин продуктов
19746,Супермаркет
19238,"Бар, паб"
17910,Салон красоты
16859,Быстрое питание
14394,Доставка еды и обедов


In [ ]:
# на основе выделенных словарей по рубрикам - лучше анализировать рестораны и кафе
interesting_rub_list = ['Кафе', 'Ресторан', 'Бар, паб', 'Быстрое питание', 'Кофейня', 'Пиццерия',
                        'Пекарня', 'Столовая', 'Суши-бар', 'Кофе с собой']

# coffee = df[df['rubrics'].isin(interesting_rub_list)] только уникальные включения ("Кафе, Кофейня" не будет)
mask = df['rubrics'].apply(lambda x: True if any(rub_x in x for rub_x in interesting_rub_list) else False)
coffee = df[mask]
coffee.index = np.arange(coffee.shape[0])
coffee.info()

NameError: ignored

## Получение координат

### Блок для экспериментов

In [ ]:
coffee.loc[:10, 'address'].apply(get_coords)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,place_id,licence,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name,boundingbox
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,206028659.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,53045167.0,55.725156150000004,37.6491671,building,yes,30.0,0.00001,building,,"4, 4-й Кожевнический переулок, Даниловский рай...","[55.7250999, 55.7252124, 37.6490376, 37.6492966]"
2,NaN,206028659.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,53045167.0,55.725156150000004,37.6491671,building,yes,30.0,0.00001,building,,"4, 4-й Кожевнический переулок, Даниловский рай...","[55.7250999, 55.7252124, 37.6490376, 37.6492966]"
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,212557584.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,72286016.0,44.64143215,41.93928223493528,building,apartments,30.0,0.00001,building,,"12, улица Павлова, Невинномысск, городской окр...","[44.6412944, 44.6415698, 41.9388146, 41.9397498]"
6,NaN,212557584.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,72286016.0,44.64143215,41.93928223493528,building,apartments,30.0,0.00001,building,,"12, улица Павлова, Невинномысск, городской окр...","[44.6412944, 44.6415698, 41.9388146, 41.9397498]"
7,NaN,373025002.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,16663096.0,47.23147470000001,39.583685911078234,building,apartments,30.0,0.00001,building,,"112, улица Ерёменко, Левенцовский, Советский р...","[47.2311138, 47.2318234, 39.5836007, 39.5846913]"
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,205621887.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,44377756.0,55.7480806,38.8442232,highway,residential,26.0,0.10001,road,улица Революции,"улица Революции, Дрезна, Орехово-Зуевский горо...","[55.7480226, 55.7480806, 38.8442232, 38.8455279]"


### Извлечение координат из данных

In [ ]:
# Функция для извлечения дат
geolocator = Nominatim(user_agent="Tester")
def get_coords(address_x):
  try:
    location = geolocator.geocode(address_x, timeout=10)
  except:
    print(address_x)
    return pd.Series([None])
  if location is None:
    # из 15/100 пропусков
    list_address = address_x.split(',')
    mask = pd.Series(['корп.' not in  i for i in list_address])
    new_address = ','.join(pd.Series(list_address)[mask].to_list())
    location = geolocator.geocode(new_address, timeout=10)
  if location is None:
    return pd.Series([None])

  return pd.Series(location.raw)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# извлечение координат из первых N элементов
N = 100
start = time.time()
сооrd = coffee.loc[:N-1, 'address'].apply(get_coords)[['osm_type', 'lat', 'lon', 'class', 'type', 'place_rank', 'importance', 'addresstype']]
end = time.time()

print(f"Время выполнения --  {round(end - start, 2)} секунд")
print(f"Количество пропусков в {N} элементах -- {сооrd['lat'].isna().sum()} / {сооrd.shape[0]}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Время выполнения --  67.01 секунд
Количество пропусков в 100 элементах -- 25 / 100


In [ ]:
# солияние df и сооrd
df_tmp = coffee.loc[:сооrd.shape[0]]
df_new = pd.concat([df_tmp, сооrd], axis=1)
df_new.sample(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,address,name_ru,rating,rubrics,text,osm_type,lat,lon,class,type,place_rank,importance,addresstype
64,"Московская область, Ленинский городской округ,...",Лесной,1.0,Ресторан;Кейтеринг,Отвратительное отношение! Перед тем как придти...,way,55.5599246,37.7127412,highway,residential,26.0,0.10001,road
7,"Ростов-на-Дону, улица Ерёменко, 112",Дело вкуса,5.0,Пекарня,Очень внимателтные и душевный продавец - пекар...,relation,47.23147470000001,39.583685911078234,building,apartments,30.0,0.00001,building
9,"Московская область, Орехово-Зуевский городской...",Пицца РИО,5.0,Кафе;Ресторан;Столовая,Очень вкусно! И по домашнему! Замечательный ко...,way,55.7480806,38.8442232,highway,residential,26.0,0.10001,road
45,"Москва, поселение Внуковское, улица Анны Ахмат...",Massimo pizza,1.0,Ресторан;Кафе,Зашли единожды в это кафе. Заказали с собой ед...,way,55.6338435,37.32415552886741,building,apartments,30.0,0.00001,building
34,"Ярославль, площадь Труда, 1",IZI kitchen,5.0,Ресторан;Кафе,Прекрасный оригинальный дизайн интерьера. Отли...,way,57.62659825,39.86546611028237,place,square,25.0,0.12501,square


In [ ]:
# вывод проблемных адресов
df_new[df_new['lat'].isna()]['address'].loc[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'Воронежская область, Богучарский район, М-4 Дон, 770-й километр'

# Предобработка отзывов

## Блок для экспериментов

In [ ]:
import pymorphy2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
morph = pymorphy2.MorphAnalyzer()

NameError: ignored

In [ ]:
# приведения слова к нормальной форме
morph.parse('мониторы')[0].normal_form

'монитор'

In [ ]:
# стоп слова
stopwords.words("russian")[:4]

['и', 'в', 'во', 'не']

## Обработка

### функции

In [ ]:
def preprocess_text(data, stop_words):
    """Очистка текста и удаление стоп-слов"""
    # замена ё на е // оставляем буквы, пробелы и знаки препинания // удаление пробелов
    text = re.sub('ё', 'е', data.lower())
    text = re.sub(r"([.,!?])", r'\1', text)
    text =re.sub(r"[^а-яА-Я\s]+", r'', text)
    text = text.strip()

    # удаление стоп слов и слов меньше длины 3
    stop_words.extend(['очень', 'самый', 'самое', 'самая'])
    text = [w for w in text.split() if w not in stop_words]
    text = [w for w in text if len(w) >= 3]

    return ' '.join(text)

def lemmatization_text(data, morph):
    result = " ".join([morph.parse(word_x)[0].normal_form
                      for word_x in data.split()])
    return result

def get_result(data, morph=pymorphy2.MorphAnalyzer(), stop_words=stopwords.words("russian")):
    if data is not np.NaN:
        result = preprocess_text(data=data, stop_words=stop_words)
        result = lemmatization_text(data=result, morph=morph)
        return result
    return ""

def transform_data(data: pd.Series) -> list:
  transform_lst = [get_result(data=i) for i in data]
  transform_lst = [x for x in transform_lst if x not in [None, '']]
  return transform_lst


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.10/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.10/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Py

### Преобразование отзывов к норм виду + CountVectorizer

In [ ]:
pyLDAvis.enable_notebook()

def theme_class(input_data, vectorizer_x=None, model_type=None):
  result = transform_data(input_data)

  if vectorizer_x is None:
    vectorizer_x = CountVectorizer(analyzer='word',
                                 min_df=10,
                                 ngram_range=(2,3),
                                 stop_words=stopwords.words("russian"))

  res_vectorizer = vectorizer_x.fit_transform(result)

  if model_type is None:
    model_type = LatentDirichletAllocation(n_components=5,
                                          learning_method='online',
                                          random_state=42,
                                          n_jobs=-1)
  model_type.fit(res_vectorizer)


  panel = pyLDAvis.lda_model.prepare(model_type,
                                     res_vectorizer,
                                     vectorizer,
                                     mds='tsne')
  print("Log Likelihood", model_type.score(res_vectorizer))
  print("Perplexity", model_type.perplexity(res_vectorizer))

  return panel, model_type, vectorizer_x

def show_topics( vectorizer_x=None, model=None, n_words=20):
  feature_names = np.array(vectorizer_x.get_feature_names_out())
  top_words = []

  for topic_weights in model.components_:
    top_keywords_locs = (-topic_weights).argsort()[:n_words]
    top_words.append(feature_names.take(top_keywords_locs))
  return top_words

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
start = time.time()
# на основе выделенных словарей по рубрикам - лучше анализировать рестораны и кафе
interesting_rub_list = ['Кафе', 'Ресторан', 'Бар, паб', 'Быстрое питание', 'Кофейня', 'Пиццерия',
                        'Пекарня', 'Столовая', 'Суши-бар', 'Кофе с собой']

# coffee = df[df['rubrics'].isin(interesting_rub_list)] только уникальные включения ("Кафе, Кофейня" не будет)
mask = df['rubrics'].apply(lambda x: True if any(rub_x in x for rub_x in interesting_rub_list) else False)
coffee = df[mask]
coffee.index = np.arange(coffee.shape[0])

vectorizer = TfidfVectorizer(analyzer='word',
# vectorizer = CountVectorizer(analyzer='word',
                             #binary=True,    # позволяет кодировать не по частоте, а просто по наличию (есть/нет слова)
                                 min_df=3,
                                 ngram_range=(2,3),
                                 stop_words=stopwords.words("russian"))


LDA_model = LatentDirichletAllocation(n_components=6,
                                          learning_method='online',
                                          random_state=42,
                                          n_jobs=-1)


N = 1000
panel_x, LDA_model, vectorizer = theme_class(coffee.loc[:N-1, 'text'],
                                             vectorizer_x=vectorizer,
                                             model_type=LDA_model)
print(f'Время обработки = {time.time() - start} секунд')
panel_x

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


BrokenProcessPool: ignored

In [ ]:
vectorizer = CountVectorizer(analyzer='word',
                                 min_df=3,
                                 ngram_range=(2,3),
                                 stop_words=stopwords.words("russian"))


LDA_model = LatentDirichletAllocation(n_components=6,
                                          learning_method='online',
                                           random_state=42,
                                          n_jobs=-1)


result = transform_data(coffee.loc[:N-1, 'text'])
res_vectorizer = vectorizer.fit_transform(result)
LDA_model.fit(res_vectorizer)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(learning_method='online', n_components=6, n_jobs=-1,
                          random_state=42)

In [ ]:
panel = pyLDAvis.lda_model.prepare(LDA_model,
                                    res_vectorizer,
                                    vectorizer)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


BrokenProcessPool: ignored

In [ ]:
pd.DataFrame(show_topics(vectorizer, LDA_model, 10))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9
0,вкусный блюдо,вкусный кухня,хороший заведение,вкусный еда,отличный обслуживание,разнообразный меню,свой дело,спасибо большой,персонал вкусный,отличный место
1,большой выбор,вежливый персонал,это заведение,это место,подача блюдо,живой музыка,посетить это,уютный атмосфера,обслуживание высота,это кафе
2,отличный место,выбор блюдо,самый вкусный,вкусный еда,рекомендовать посещение,приятный персонал,быстрый обслуживание,готовить вкусно,выше среднее,вкусно быстро
3,персонал вежливый,еда вкусный,уютный место,выше всякий,центр город,выше всякий похвала,всякий похвала,рекомендовать посетить,хороший уютный,вкусный цена
4,живой музыка,высокий уровень,вкусный еда,приветливый персонал,детский комната,вкусный пицца,это просто,спасибо огромный,обслуживание вкусный,персонал приветливый
5,день рождение,отмечать день,кухня вкусный,отмечать день рождение,данный заведение,прекрасный обслуживание,оставлять желать,сделать заказ,уютный кафе,провести время
